In [ ]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
import csv
import random
from collections import Counter

In [ ]:
#load the word2vec-google-news-300 pretrained embedding model
model = api.load("word2vec-google-news-300")

In [ ]:
#check if guess
def is_guess(line):
    if not model.has_index_for(line['question']):
        return True
    for i in range(4):
        if model.has_index_for(line[str(i)]):
            return False
    return True

In [ ]:
#find the closest synonym to the question-word. Use the similarity method from Gensim.
output_data = []
with open('synonyms.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    most_similar = 0
    guess_word = ""
    label = ""
    for line in csv_reader:
        for i in range(4):
            if (is_guess(line)):
                random_word_key = random.randint(0,3)
                guess_word = line[str(random_word_key)]
                label = "guess"
                break
            if not model.has_index_for(line[str(i)]):
                continue
            else:
                model_similarity = model.similarity(line['question'], line[str(i)])
            if (model_similarity > most_similar):
                most_similar = model_similarity
                guess_word = line[str(i)]
        if (label != "guess"):
            label = "correct" if (guess_word == line['answer']) else "wrong"
        
        output_data.append({"question": line['question'], "answer": line['answer'], "guess": guess_word, "label": label})
        most_similar = 0
        guess_word = ""
        label = ""

In [ ]:
#write results to csv file
with open('word2vec-google-news-300-details.csv', 'w') as new_file:
    fieldnames = output_data[0].keys()
    csv_writer = csv.DictWriter(new_file, fieldnames, lineterminator = '\n')
    csv_writer.writeheader()
    csv_writer.writerows(output_data)